# 🌊 Diffusion Sudoku Training (Iterative Denoising)

Train a Sudoku solver using the **Diffusion Depth Controller** with iterative denoising.

**Key Features:**
- Iterative denoising of routing weights
- Learned noise schedules (linear, cosine, sqrt)
- Adaptive LayerNorm (adaLN) conditioning from DiT
- Smooth, temporally coherent routing evolution

**Reference:** [Scalable Diffusion Models with Transformers (DiT)](https://arxiv.org/abs/2212.09748)


In [1]:
# Clone repository and install dependencies
!git clone https://github.com/Eran-BA/PoT.git
%cd PoT
!pip install -q -r requirements.txt


Cloning into 'PoT'...
remote: Enumerating objects: 2637, done.
remote: Counting objects: 100% (166/166), done.
remote: Compressing objects: 100% (108/108), done.
remote: Total 2637 (delta 110), reused 107 (delta 58), pack-reused 2471 (from 2)
Receiving objects: 100% (2637/2637), 2.42 MiB | 23.11 MiB/s, done.
Resolving deltas: 100% (1667/1667), done.
/content/PoT
ERROR: Could not find a version that satisfies the requirement torch==2.0.1 (from versions: 2.2.0, 2.2.1, 2.2.2, 2.3.0, 2.3.1, 2.4.0, 2.4.1, 2.5.0, 2.5.1, 2.6.0, 2.7.0, 2.7.1, 2.8.0, 2.9.0, 2.9.1)
ERROR: No matching distribution found for torch==2.0.1


In [2]:
# Check GPU and verify Diffusion controller
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

from src.pot.core import DiffusionDepthController, create_controller
print("\nDiffusion controller available: ✓")

# Quick test
controller = create_controller("diffusion", d_model=256, n_heads=8, noise_schedule="cosine")
print(f"Controller params: {sum(p.numel() for p in controller.parameters()):,}")

# Test denoising step
X = torch.randn(2, 81, 256)
alpha, state, aux = controller.step(X)
print(f"Initial sigma (noise): {aux['sigma']:.4f}")


PyTorch version: 2.9.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
Memory: 42.5 GB

Diffusion controller available: ✓
Controller params: 2,567,176
Initial sigma (noise): 1.0000


In [3]:
!git pull origin main

From https://github.com/Eran-BA/PoT
 * branch            main       -> FETCH_HEAD
Already up to date.


In [4]:
# W&B Login
import wandb
wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eranbt92 (eranbt92-open-university-of-israel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

## 🚀 Train with Diffusion Controller


In [ ]:
# Run Diffusion training with all arguments
!python experiments/sudoku_poh_benchmark.py \
    --download \
    --model hybrid \
    --controller diffusion \
    --d-model 512 \
    --d-ff 2048 \
    --n-heads 8 \
    --H-layers 2 \
    --L-layers 2 \
    --H-cycles 2 \
    --L-cycles 6 \
    --halt-max-steps 4 \
    --halt-exploration-prob 0.1 \
    --max-depth 32 \
    --hrm-grad-style \
    --epochs 500 \
    --batch-size 128 \
    --lr 3e-4 \
    --weight-decay 0.01 \
    --beta1 0.9 \
    --beta2 0.95 \
    --warmup-steps 500 \
    --lr-min-ratio 0.1 \
    --grad-clip 1.0 \
    --dropout 0.0 \
    --subsample 10000 \
    --num-aug 100 \
    --eval-interval 50 \
    --wandb \
    --project sudoku-diffusion \
    --seed 42 \
    --output experiments/results/diffusion_sudoku


Device: cuda
GPU: NVIDIA A100-SXM4-40GB
Note: Augmentation is now ON-THE-FLY (not pre-computed)
train.csv: 100% 719M/719M [00:02<00:00, 308MB/s]
  Train puzzles: 9000, Val puzzles: 1000
Processing train: 100% 9000/9000 [00:00<00:00, 35093.92it/s]
  train: 9000 puzzles (augmentation: on-the-fly)
Processing val: 100% 1000/1000 [00:00<00:00, 35173.54it/s]
  val: 1000 puzzles (augmentation: on-the-fly)
test.csv: 100% 79.4M/79.4M [00:00<00:00, 95.4MB/s]
Processing test: 422786it [00:12, 33213.29it/s]
  test: 422786 puzzles
✓ Dataset saved to data/sudoku-extreme-10k-aug-100
[train] Loaded 9000 puzzles
  Augmentation: ON-THE-FLY
[val] Loaded 1000 puzzles
  Augmentation: OFF
Using VAL split (held-out training puzzles) for evaluation
Hybrid model: H_cycles=2, L_cycles=6
H_layers=2, L_layers=2, dropout=0.0
Controller: diffusion
Gradient style: HRM (last L+H only)
ACT enabled: halt_max_steps=4, exploration=0.1

Model: HYBRID
Parameters: 37,335,580 (37.34M)
wandb: Currently logged in as: eranbt92 

## 📈 Results


In [ ]:
# Plot results
import json
import matplotlib.pyplot as plt

with open('experiments/results/diffusion_sudoku/hybrid_results.json', 'r') as f:
    results = json.load(f)

print(f"Best Grid Accuracy: {results['best_grid_acc']:.2f}%")
print(f"Parameters: {results['parameters']:,}")

history = results['history']
epochs = [h['epoch'] for h in history]
train_acc = [h['train_grid_acc'] for h in history]
test_acc = [h['test_grid_acc'] for h in history]

plt.figure(figsize=(10, 5))
plt.plot(epochs, train_acc, label='Train')
plt.plot(epochs, test_acc, label='Test')
plt.xlabel('Epoch')
plt.ylabel('Grid Accuracy (%)')
plt.title('Diffusion Controller - Sudoku Training')
plt.legend()
plt.grid(True)
plt.show()


## 🔬 Visualize Noise Schedule


In [ ]:
# Visualize sigma (noise level) across depth steps
from src.pot.core import DiffusionDepthController

controller = DiffusionDepthController(d_model=256, n_heads=8, max_depth=32, noise_schedule="cosine")
sigmas = controller.sigma_schedule.cpu().numpy()

plt.figure(figsize=(10, 4))
plt.plot(range(len(sigmas)), sigmas, 'b-', linewidth=2)
plt.xlabel('Depth Step')
plt.ylabel('Sigma (Noise Level)')
plt.title('Cosine Noise Schedule - Denoising Process')
plt.grid(True)
plt.show()

print(f"Step 0 (start):  σ = {sigmas[0]:.4f} (noisy)")
print(f"Step 16 (mid):   σ = {sigmas[16]:.4f}")
print(f"Step 31 (end):   σ = {sigmas[31]:.4f} (clean)")
